# 논문에서 말하는 임베딩 조정이란?

일반적으로, 단어 임베딩은 단어나 구를 고정된 길이의 벡터로 변환하는 작업.

하지만, 동일한 의미를 가지는 단어(또는 약물)가 여러 이름으로 표현될 수도 있음.

예를 들어, 우울증 치료에 사용되는 약물인 'fluoxetine(플루옥세틴)'은 상표명 'Prozac(프로작)'으로도 잘 알려져 있음.

만약 모델이 이 두 이름을 서로 다른 것으로 처리한다면, 모델의 성능이 떨어질 수 있기에 두 이름의 임베딩을 평균 풀링하여 동일한 벡터로 조정하는 방법을 채택.

## 임베딩 조정 과정

1. 임베딩 벡터 이해
   - 임베딩 벡터는 모델이 텍스트의 의미를 이해하는 데 사용하는 수치 표현. 각 단어는 모델 내부에서 고정된 길이의 벡터로 표현.
   - 예를 들어, BERT 모델의 경우 각 단어는 768차원의 벡터로 표현될 수 있음.
2. 어휘 확장
   - 모델의 기존 어휘에 새로운 단어를 추가할 수 있음. 예를 들어, 'fluoxetine'과 'Prozac'이라는 새로운 단어를 추가.
   - `tokenizer.add_tokens`를 사용하여 새로운 단어를 모델의 어휘에 추가.
   - `model.resize_token_embeddings`를 사용하여 모델의 임베딩 층을 새로운 어휘 크기에 맞게 조정.
3. 임베딩 초기화 및 추출
   - 추가된 단어들의 임베딩 벡터는 초기화되거나 기존 임베딩 벡터의 평균으로 설정.
   - 각 추가된 단어(예: 'fluoxetine', 'Prozac')의 임베딩 벡터를 추출.
4. 평균 풀링(Averaging Embeddings)
   - 동일한 약물을 의미하는 여러 단어들의 임베딩 벡터를 평균함.
   - 예를 들어, 'fluoxetine'과 'Prozac'의 임베딩 벡터를 평균하여 새로운 임베딩 벡터를 만듦. 이를 통해 이 두 단어는 동일한 벡터로 표현됨.
5. 임베딩 재할당
   - 평균화된 임베딩 벡터를 각 단어의 임베딩 벡터로 재할당함.
   - 이을 통해, 모델은 'fluoxetine'과 'Prozac'을 같은 의미로 인식하게 됨.

정리하자면, 동일한 개념을 가진 여러 표현(상표의 보통명사화 등)을 모델이 동일하게 인식하도록 조정(각 임베딩 벡터를 평균 풀링)

ex) 반창고(원래 이름)와 대일밴드(상표명)가 동일한 임베딩 벡터를 갖도록 조정

### 예시 코드

In [1]:
import torch
import numpy as np
import pandas as pd
import re

from konlpy.tag import Okt

from transformers import BertTokenizer, BertModel
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

%reload_ext autotime

time: 0 ns (started: 2024-06-11 20:59:22 +09:00)


In [2]:
# 사전 학습된 BERT 모델과 토크나이저 로드
model_name = 'snunlp/KR-BERT-char16424'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# 반창고와 대일밴드를 모델의 어휘에 추가
new_words = ['반창고', '대일밴드'] # 원래는 tokenizer.convert_tokens_to_ids()에 '반창고'와 '대일밴드'를 넣으면 1이 나옴(<UNK>).
tokenizer.add_tokens(new_words)
model.resize_token_embeddings(len(tokenizer))

# 추가된 단어의 임베딩 초기화 또는 기존 임베딩 평균화
def get_embedding(word: str) -> np.ndarray:
    input_ids = torch.tensor(tokenizer.encode(word, add_special_tokens=False)).unsqueeze(0)
    with torch.no_grad():
        outputs = model(input_ids)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

# 각 단어의 임베딩 추출
plastic_bandgage_embeddings = get_embedding('반창고')
daeil_band_embeddings = get_embedding('대일밴드')

time: 1.86 s (started: 2024-06-11 20:59:25 +09:00)


In [3]:
for word in ['반창고', '대일밴드']:
    print(tokenizer.convert_tokens_to_ids(word))

16424
16425
time: 0 ns (started: 2024-06-11 20:59:27 +09:00)


In [4]:
# 두 임베딩 벡터 출력 및 차이 확인
print("반창고 임베딩 벡터:")
print(plastic_bandgage_embeddings)

print("대일밴드 임베딩 벡터:")
print(daeil_band_embeddings)

# 두 벡터 간의 차이 계산 (유클리드 거리)
distance = torch.dist(torch.tensor(plastic_bandgage_embeddings), torch.tensor(daeil_band_embeddings)).item()
print(f"Distance between '반창고' and '대일밴드' embeddings: {distance}")

반창고 임베딩 벡터:
[-2.51885384e-01 -4.39611167e-01 -4.64528143e-01  1.22299530e-02
 -1.11204052e+00  1.85486600e-01 -5.41931055e-02 -3.47325295e-01
  3.44963670e-01 -5.70219398e-01 -3.64010274e-01 -6.06023371e-02
  2.19425231e-01 -2.74497062e-01 -1.52420118e-01  5.57348877e-02
 -2.10203871e-01  2.80949265e-01 -7.81457052e-02 -4.99139912e-02
 -3.37307245e-01 -3.58521305e-02  7.02040136e-01  1.59353502e-02
 -1.08070955e-01  6.89583778e-01  9.89101410e-01 -2.87397712e-01
 -3.25210065e-01 -7.29681134e-01  1.12842786e+00 -4.65413742e-02
  5.09656012e-01 -9.66447815e-02 -6.24305308e-01 -2.38738567e-01
 -6.26215875e-01 -1.33111089e-01 -3.92915547e-01 -6.41944587e-01
  1.59939900e-01 -2.99864471e-01 -1.06815487e-01 -2.61866450e-01
 -3.40269655e-01  7.90566579e-02  4.82009500e-01  2.76144326e-01
 -3.21305931e-01 -5.55042148e-01  8.09841007e-02 -1.29980534e-01
 -3.36739302e-01 -1.09935868e+00 -8.55732262e-02 -1.76296353e-01
  1.15705550e-01  3.16138923e-01 -3.66336733e-01  4.79935169e-01
 -5.18146932e

In [5]:
# 임베딩 평균화
average_embedding = (plastic_bandgage_embeddings + daeil_band_embeddings) / 2

# 모델의 임베딩 벡터 재설정
word_to_index = tokenizer.convert_tokens_to_ids(new_words) # [16424, 16425]

time: 0 ns (started: 2024-06-11 20:59:36 +09:00)


In [6]:
# 임베딩 벡터를 평균화된 벡터로 설정
model.get_input_embeddings().weight.data[word_to_index[0]] = torch.tensor(average_embedding)
model.get_input_embeddings().weight.data[word_to_index[1]] = torch.tensor(average_embedding)

print("임베딩 조정 후 반창고 임베딩 벡터:")
print(model.get_input_embeddings().weight.data[word_to_index[0]])
print("임베딩 조정 후 대일밴드 임베딩 벡터:")
print(model.get_input_embeddings().weight.data[word_to_index[1]])

임베딩 조정 후 반창고 임베딩 벡터:
tensor([-8.7724e-02, -4.1309e-01, -2.9472e-01, -1.1762e-01, -1.0329e+00,
         3.1249e-01, -9.2931e-02, -3.4079e-01,  3.7619e-01, -5.9808e-01,
        -5.6911e-01, -2.0154e-01,  3.0685e-01, -2.5243e-01, -2.8629e-01,
        -4.2835e-03, -2.0449e-01,  1.3335e-01, -3.1639e-02,  1.7507e-01,
        -2.8447e-01, -8.3718e-02,  5.3158e-01, -3.3488e-02, -4.3037e-02,
         5.4671e-01,  9.7631e-01, -2.6103e-01, -1.7637e-01, -1.0079e+00,
         1.2169e+00, -5.8517e-02,  4.4193e-01, -1.8191e-01, -6.0324e-01,
        -1.2752e-01, -7.3071e-01, -9.8653e-03, -4.3463e-01, -6.4039e-01,
        -5.7045e-03, -2.3052e-01,  2.5356e-02, -2.0860e-01, -3.8265e-01,
         8.1004e-02,  3.3484e-01,  2.2979e-01, -2.4003e-01, -4.4011e-01,
        -4.8524e-02, -1.1025e-01, -2.2244e-01, -1.1880e+00, -1.3381e-01,
        -1.7997e-01,  1.0246e-01,  2.8174e-01, -3.1707e-01,  2.8463e-01,
        -5.2294e-01, -4.3247e-01,  5.7454e-01, -1.4324e-01,  9.5267e-02,
         7.1026e-02, -3.8592e-

## 평균 풀링 외 다른 기법들

평균 풀링은 구현이 간단하고 직관적이지만, 모든 단어를 동일한 중요도로 처리하기 때문에 더 중요한 단어가 있을 경우에 적합하지 않을 수 있음.

In [52]:
fluoxetine_embedding = np.array([0.1, 0.2, 0.3])
prozac_embedding = np.array([0.15, 0.25, 0.35])

average_embedding = (fluoxetine_embedding + prozac_embedding) / 2
print('평균 풀링 임베딩:', average_embedding)

평균 풀링 임베딩: [0.125 0.225 0.325]
time: 0 ns (started: 2024-06-11 17:30:20 +09:00)


### 가중 평균 풀링(Weighted Average Pooling)

중요한 단어가 더 큰 영향을 미치게 할 수 있지만, 각 단어의 중요도를 결정하는 것이 어려울 수 있음.

In [53]:
weights = np.array([0.6, 0.4])
weighted_average_embedding = (fluoxetine_embedding * weights[0] + prozac_embedding * weights[1]) / 2
print('가중 평균 임베딩:', weighted_average_embedding)

가중 평균 임베딩: [0.06 0.11 0.16]
time: 0 ns (started: 2024-06-11 17:30:20 +09:00)


### 최소/최대 풀링(Min/Max Pooling)

특정 차원에서 극단적인 값을 유지하여 정보 손실을 최소화할 수 있으나, 덜 직관적.

In [54]:
min_embedding = np.minimum(fluoxetine_embedding, prozac_embedding)
max_embedding = np.maximum(fluoxetine_embedding, prozac_embedding)
print('최소 임베딩:', min_embedding)
print('최대 임베딩:', max_embedding)

최소 임베딩: [0.1 0.2 0.3]
최대 임베딩: [0.15 0.25 0.35]
time: 0 ns (started: 2024-06-11 17:30:20 +09:00)


### 거리 기반 클러스터링(Distance-Based Clustering)

의미적으로 가까운 단어들을 그룹화

In [55]:
embeddings = np.array([fluoxetine_embedding, prozac_embedding])
kmeans = KMeans(n_clusters=1)
kmeans.fit(embeddings)

cluster_center = kmeans.cluster_centers_[0]
print('거리 기반 임베딩:', cluster_center)

거리 기반 임베딩: [0.125 0.225 0.325]
time: 47 ms (started: 2024-06-11 17:30:20 +09:00)


c:\ProgramData\Anaconda3\envs\myenv\lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


### PCA

In [56]:
fluoxetine_embedding = np.array([0.1, 0.2, 0.3])
prozac_embedding = np.array([0.15, 0.25, 0.35])
other_embedding = np.array([0.12, 0.22, 0.32])

embeddings = np.array([fluoxetine_embedding, prozac_embedding, other_embedding])

pca = PCA(n_components=1)
pca_embeddings = pca.fit_transform(embeddings)

principal_component = pca_embeddings.flatten()
print('PCA 임베딩:', principal_component)

PCA 임베딩: [-0.04041452  0.04618802 -0.0057735 ]
time: 0 ns (started: 2024-06-11 17:30:20 +09:00)


## 그래서 어떻게 적용하면 좋을까?

크게 두 가지로 생각함.
1. 학습데이터셋에 있는 단어 중 단어 사전에 없는 단어를 추가하고 동의어끼리 임베딩 조정.
   - 위 '반창고'와 '대일밴드'의 예시처럼 진행하면 됨.
2. 단어사전에 있는 단어에 대해서만 적용 -> 그런데 그럴 필요가 있을까?
   
   Why? 잘 구축된 임베딩 공간에서는 동의어가 비슷한 단어 벡터로 임베딩될 것으로 생각

### 학습데이터셋 중 단어 사전에 없는 단어를 추가하고 동의어끼리 임베딩 조정

학습데이터셋('./data/train.csv')에 있는 단어를 하나씩 보면서 KR-BERT에 없는 단어를 찾아보자.

In [57]:
file_path = './data/train.csv'
df = pd.read_csv(file_path)

dialogues = df['HS'].dropna().tolist()

okt = Okt()

model_name = 'snunlp/KR-BERT-char16424'
tokenizer = BertTokenizer.from_pretrained(model_name)

vocab_set = set(tokenizer.vocab.keys())

# 불용어
stopwords = ['은', '는', '이', '가', '을', '를', '에', '의', '도', '으로', '에서', '와', '과', '한', '하다']

# 어휘에 없는 단어들을 수집할 리스트
missing_words = set()

# 정규 표현식으로 문장부호 제거
def clean_text(text: str) -> str:
    return re.sub(r'[^\w\s]', '', text)

for dialogue in dialogues:
    # 문장부호 제거 및 텍스트 정리
    cleaned_text = clean_text(dialogue)
    
    # 형태소 분석을 통해 명사와 동사 추출
    words = okt.morphs(cleaned_text, stem=True)  # 어간 추출을 포함한 형태소 분석
    
    # 불용어 제거
    filtered_words = [word for word in words if word not in stopwords]
    
    # 단어가 토크나이저의 어휘에 없는지 확인
    for word in filtered_words:
        if word not in vocab_set:
            missing_words.add(word)

# 결과 출력
print("단어장에 없는 단어들:")
print(missing_words) # 12,607개

단어장에 없는 단어들:
{'떵떵', '귀지', '구설', '팔자주름', '연애편지', '협방', '진하다', '나얘', '겪다', '세련되다', '불효', '육체', '도돌이표', '캐드', '궤양', '무궁무진', '포도', '헛일', '기침', '저런', '부간', '유명인', '전기자동차', '알츠하이머병', '표준어', '죽도', '밤샘', '다가오다', '과민', '수의', '양사', '탕감', '소지품', '낙제', '총대', '블라우스', '쇠약하다', '늦다', '건데전', '파손', '퇴로', '아로마', '오에이', '속속', '비대', '전복죽', '줄걸', '기재', '생활용품', '인센티브', '감쪽', '꽃꽂이', '시간제', '융화', '웅덩이', '손자', '여대', '감싸다', '부착', '도끼', '주례', '난폭하다', '코로나바이러스', '갑작스레', '노골', '정리해고', '폐인', '불이익', '꽁무니', '굴복', '인문', '의리', '관계없이', '유비', '남다르다', '실비', '러닝머신', '의과대학', '오류로', '등하교', '회서', '악한', '경쟁률', '위주', '수식', '엄가', '해인사', '후생', '종친회', '물러서다', '시니어', '최상', '종신', '성차별', '오만원', '기척', '여자애', '노점', '피멍', '통탄', '과는', '사주팔자', '서인', '나진', '원동력', '부자', '깨끗해지다', '비일비재하다', '거두다', '무뎌', '예년', '아가', '모색', '파렴치하다', '주니', '임종이', '들추다', '가지가지', '해홈', '공고', '기억상실증', '이견', '시집가다', '트릴', '부숴', '안건', '하기는', '부친', '건실', '초심', '만화가', '이지함', '고다', '이제야', '해학', '올려놓다', '으론', '따윈', '뉘앙스', '프러포즈', '치근덕거리다', '아이미', '희한하다', '명확하다', '양화', '분윳값

생각보다 많음.

또 다시 생각을 해보니, 단어사전에 있으면 단어에 해당하는 임베딩 벡터를 가져오고, 없으면 추가해주면 된다.

하지만, 동의어 리스트를 만들어 주어야 함.

그런데, 사실 임베딩 조정이라는 작업이 단어 사전에 없는 특정 단어들이 같은 단어로 인식하게끔 하는 것이라 밑의 예시는 적절하지 않음.

그리고 우리는 논문 내용처럼 의약품을 다루는 것이 아닌, 일반 발화문이기에 임베딩 조정이 필요할까? 라는 생각도 들었음.

In [89]:
synonym_groups = {
    '치료제': ['치료제', '약', '의약품'],
    '컴퓨터': ['컴퓨터', 'PC', '노트북'],
    '치매': ['알츠하이머병', '...']
}

model_name = 'snunlp/KR-BERT-char16424'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

vocab_set = set(tokenizer.vocab.keys())

missing_words = set()

for group in synonym_groups.values():
    for word in group:
        if word not in vocab_set:
            missing_words.add(word)
            
missing_words = list(missing_words)
tokenizer.add_tokens(missing_words)
model.resize_token_embeddings(len(tokenizer))

# 각 동의어 그룹에 대해 임베딩 벡터를 조정
for group_key, group_words in synonym_groups.items():
    # 동의어 그룹의 임베딩 벡터를 수집
    group_embeddings = []
    for word in group_words:
        if word in vocab_set:
            embedding_vector = model.get_input_embeddings().weight[tokenizer.convert_tokens_to_ids(word)].detach().numpy()
        else:
            embedding_vector = torch.randn(model.config.hidden_size).numpy()  # 새로운 임베딩은 랜덤 초기화
        group_embeddings.append(embedding_vector)
    
    # 평균 풀링을 통해 임베딩 벡터를 결합
    averaged_embedding = np.mean(group_embeddings, axis=0)
    
    # 결합된 임베딩 벡터를 각 단어에 할당
    for word in group_words:
        if word in missing_words:
            model.get_input_embeddings().weight.data[tokenizer.convert_tokens_to_ids(word)] = torch.tensor(averaged_embedding, dtype=torch.float32)
            

time: 2.69 s (started: 2024-06-11 17:39:16 +09:00)


만약, 진행한다면 동의어 리스트가 필요한데 이를 어떻게 해결할까?

위에서 단어 사전에 없는 단어들 12,607개에 대해 Sentence Transformer?로 동의어 뽑아서 리스트 만들고 진행하는 건 어떨까..

또는 수작업

In [100]:
data = pd.read_csv('./data/intermediate/mental_trainset.csv')

data

,Unnamed: 0,book_id,category,popularity,text,word_segment,publication_ymd
0,86,MPA000107,정신과학,2,"이하, 실시예를 첨부된 도면을 참조하여 상세하게 설명한다. 도 1은 학습자의 뇌파를...",1843,20171204
1,219,MPA000269,정신과학,2,"본 발명의 이점 및 특징, 그리고 그것들을 달성하는 방법은 첨부되는 도면과 함께 상...",1759,20130805
2,259,MPA000320,정신과학,2,"이하, 첨부한 도면을 참조하여 본 발명의 실시예에 대하여 당업자가 용이하게 실시할 ...",1993,20150501
3,321,MPA000395,정신과학,3,"이하, 첨부된 도면을 참조하여 본 발명에 의한 두피와 두개골 사이의 음파전달 경계조...",1511,20110125
4,572,MPA000699,정신과학,2,이하에서는 첨부한 도면을 참조하여 본 발명을 설명하기로 한다. 그러나 본 발명은 여...,1923,20190308
...,...,...,...,...,...,...,...
386,37140,MTB026460,정신과학,2,노인평가도구 (Geriatric Assessment Tools) 7.1 시각장애(v...,795,20150831
387,37165,MTB026490,정신과학,3,"불안 장애에 포함되는 질환들은 공포, 불안, 또는 회피 행동이 발생하는 대상이나,...",520,20160222
388,37245,MTB026586,정신과학,3,② 기능적 평가 재활 서비스를 제공하기 위해서는 환자에게 남아 있는 기능과 결손에 ...,766,20160222
389,37291,MTB026640,정신과학,2,기면병의 진단에는 수면 다원 검사 및 수면 잠복기 반복 검사가 필수검사이다. 다음은...,711,20200220


time: 63 ms (started: 2024-06-11 17:41:13 +09:00)
